In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F


from torch.utils.data import Dataset, DataLoader

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.RandomCrop((256,256),pad_if_needed=True),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.OxfordIIITPet("pets_data/","trainval",transform=transform,download=True)
test_dataset = torchvision.datasets.OxfordIIITPet("pets_data/","test",transform=transform,download=True)

train_loader = DataLoader(train_dataset,batch_size=8)
test_loader = DataLoader(test_dataset,batch_size=8)

##### This network design is too shallow for the task, your job is to make it deeper! #####

class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5, padding=2)  # 5x5 convolution
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)  # 3x3 convolution
        self.pool1 = nn.MaxPool2d(2, 2)  # 2x2 MaxPool
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)  # 3x3 convolution
        self.conv4 = nn.Conv2d(256, 256, 3, padding=1)  # 3x3 convolution
        self.pool2 = nn.MaxPool2d(2, 2)  # 2x2 MaxPool
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)  # 3x3 convolution
        self.pool3 = nn.MaxPool2d(2, 2)  # 2x2 MaxPool
        self.conv6 = nn.Conv2d(512, 512, 3, padding=1)  # 3x3 convolution
        self.fc1 = nn.Linear(512 * 32 * 32, 37)  # Linear layer with the final image size (32x32 here)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        x = F.relu(self.conv5(x))
        x = self.pool3(x)
        x = F.relu(self.conv6(x))
        x = x.view(x.size(0), -1)  # Flatten the tensor for the linear layer
        x = self.fc1(x)
        return x

#######################


model = MyNetwork()

##### Training Loop #####

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_func = torch.nn.CrossEntropyLoss()

num_epochs = 5

for epoch in range(num_epochs):
    print("Epoch:", epoch)
    for im, label, in train_loader:

        optimizer.zero_grad()

        output = model(im)
        loss = loss_func(output,label)
        print(loss)

        loss.backward()
        optimizer.step()


##### Testing Loop #####

correct = 0; total=0

model.eval()
with torch.no_grad():
    for im,label in test_loader:
        
        output = model(im)
        guess = torch.argmax(output,dim=1)
        
        correct += torch.sum(guess == label).item()
        total += len(guess)

print("Testing Accuracy is", correct/total * 100, "%")

Epoch: 0
tensor(3.6092, grad_fn=<NllLossBackward0>)
tensor(3.1357, grad_fn=<NllLossBackward0>)
tensor(2.2898, grad_fn=<NllLossBackward0>)
tensor(1.0055, grad_fn=<NllLossBackward0>)
tensor(0.1435, grad_fn=<NllLossBackward0>)
tensor(0.0016, grad_fn=<NllLossBackward0>)
tensor(16.3195, grad_fn=<NllLossBackward0>)
tensor(16.3047, grad_fn=<NllLossBackward0>)
tensor(10.2583, grad_fn=<NllLossBackward0>)
tensor(5.7461, grad_fn=<NllLossBackward0>)
tensor(3.7069, grad_fn=<NllLossBackward0>)
tensor(2.4582, grad_fn=<NllLossBackward0>)
tensor(3.2171, grad_fn=<NllLossBackward0>)
tensor(3.8768, grad_fn=<NllLossBackward0>)
tensor(3.5339, grad_fn=<NllLossBackward0>)
tensor(3.2894, grad_fn=<NllLossBackward0>)
tensor(3.1029, grad_fn=<NllLossBackward0>)
tensor(2.9499, grad_fn=<NllLossBackward0>)
tensor(3.0718, grad_fn=<NllLossBackward0>)
tensor(3.7564, grad_fn=<NllLossBackward0>)
tensor(3.6223, grad_fn=<NllLossBackward0>)
tensor(3.4818, grad_fn=<NllLossBackward0>)
tensor(3.3430, grad_fn=<NllLossBackward0>)